In [1]:
from tqdm import tqdm
import time
from random import randint
from langame.langame_client import LangameClient
import random
import openai
import websocket
import json
from pprint import pprint

c = LangameClient("../config.yaml")
memes = []
for e in c._firestore_client.collection('memes').stream():
    memes.append((e.id, e.to_dict()))
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
def get_prompt(topics):
    rnd = random.choices(memes, k=500)
    return [(e[1]['content'], e[1]['topics']) for e in rnd if len(intersection(e[1]['topics'], topics)) > 0]


In [26]:
def get_random_meme():
    rnd = random.choices(memes, k=500)
    return rnd[0][1]['content']
get_random_meme()

"You're stranded on an island with a perfect tree house tree positioned exactly right where the tree house would be so no one would notice if there was a tree house inside. You can't see it from where you are below. But someone else who's also stuck on this island with you can see it. But they can't reach it and can't tell if there's anything inside or not. What do you do?"

In [88]:
def build_conversational_prompt(history):
    # remove first [\n
    formatted_history = json.dumps(history, indent=4).replace("[\n", "")
    nfh = ""
    for e in formatted_history.split("},")[:-1]:
        # add "# This is the answer of Bob to Alice" between every object
        nfh += e + "},\n" + f"\t# This is the answer to the user" # TODO:  {'Alice' if 'Bob' in e else 'Bob'} to {'Bob' if 'Bob' in e else 'Alice'}
    starters = f"""# This is a list of conversation starters with format (conversation starter, [list of topics])
conversation_starters = {get_prompt(['philosophy'])[:10]}

# This is a conversation between two people
one_to_one_conversation = [
{nfh},"""
    # replace last \n], by , to indicate that GPT3 should keep filling this list
    return starters[:-1]

import re
p = build_conversational_prompt([
        {"user": "Bob", "message": "Hi Alice"},
        {"user": "Alice", "message": "Hi Bob"},
        {"user":"Bob",
        "message":f"Listen Alice, I will be honest, I want to try to have more meaningful conversations, I want to ask you a deep question so we skip the small talk, '{get_random_meme()}'"},
    ])
message = openai.Completion.create(
    engine="davinci-codex",
    prompt=p,
    temperature=1,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0.7,
    presence_penalty=0,
    stop=["},"]
)["choices"][0]["text"]

#re.compile(r'["\']message["\']:.*["\'](.*?)["\']').findall(message)[0]
p

'# This is a list of conversation starters with format (conversation starter, [list of topics])\nconversation_starters = [(\'Why are there more women studying philosophy now than men?\', [\'philosophy\']), (\'What kind of influence did Socrates clearly have over his disciples along our cultural values. Shall we revive this today or shall it remain just a nostalgic thought for eternity ? \', [\'philosophy\']), (\'What is the role of philosophy in human society?\', [\'philosophy\', \'knowledge\', \'politic\']), (\'There must be something else that gives our systems the meaning we give to it. What other meaning could we use to find meaning in life?\', [\'philosophy\']), (\'What do you think about the possibility of extraterrestrial life? How do you imagine it would be different from human life?\', [\'physic\', \'philosophy\']), (\'If all things are continuous do not exist what makes them separate ?\', [\'mathematic\', \'philosophy\']), (\' Does human rationality have limits? Are there thi

In [42]:
def build_message(message: str) -> str:
    return json.dumps({"text": message})

In [91]:
from datetime import datetime
history = [
  {"user": "Bob", "message": "Hi Alice"},
  {"user": "Alice", "message": "Hi Bob"},
  {"user":"Bob",
  "message":f"Listen Alice, I will be honest, I want to try to have more meaningful conversations, I want to ask you a deep question so we skip the small talk, '{get_random_meme()}'"},
]
max_failure, failures = 5, 0
text_to_send = []
ws = websocket.WebSocket()
ws.connect("ws://localhost:8080")
result = ""
while "[DONE]" not in result:
  print("sending begin")
  ws.send(build_message("begin"))
  result = ws.recv()
  time.sleep(4)
print("conversation between gpt3 (Bob) and bb2 (Alice) started!")
ws.send(build_message("Hi Alice"))
result = ws.recv()
ws.send(build_message(history[-1]["message"]))
for i in range(10**12):
  try:
    result = ws.recv()
    bb2_response = json.loads(result)["text"]
    history.append({"time": str(datetime.now()), "user": "Alice", "message": json.loads(result)["text"]})
    with open("generated_samples.txt", "a") as f:
      f.write(json.dumps(history[-1]) + "\n")

    print("\n~~~~HISTORY~~~~\n")
    print(json.dumps(history, indent=4))
    print("\n~~~~HISTORY~~~~\n")

    # maximum size
    if len(history) > 20:
      history.pop(0)
    prompt = build_conversational_prompt(history)
    print("\n~~~~PROMPT~~~~\n")
    print(prompt)
    print("\n~~~~PROMPT~~~~\n")
    response = openai.Completion.create(
      engine="davinci-codex",
      prompt=prompt,
      temperature=1,
      max_tokens=300,
      top_p=1,
      frequency_penalty=0.9,
      presence_penalty=0,
      stop=["},"]
    )
    # this is a regex to extract the "message" part of the json
    regex = re.compile(r'["\']message["\']:.*["\'](.*?)["\']')
    text_to_send = regex.findall(message)
    if len(text_to_send) == 0:
      print("skipping, invalid json")
      continue
  except Exception as e:
    failures += 1
    if failures > max_failure:
      print("too many failures")
      break
    print(e)
    continue
  if response["choices"][0]["finish_reason"] == "length":
    continue
  history.append({"time": str(datetime.now()), "user": "Bob", "message": text_to_send[0]})
  with open("generated_samples.txt", "a") as f:
    f.write(json.dumps(history[-1]) + "\n")
  time.sleep(randint(1,3))
  ws.send(json.dumps({"text": text_to_send[0]}))
  

sending begin
sending begin
conversation between gpt3 (Bob) and bb2 (Alice) started!

~~~~HISTORY~~~~

[
    {
        "user": "Bob",
        "message": "Hi Alice"
    },
    {
        "user": "Alice",
        "message": "Hi Bob"
    },
    {
        "user": "Bob",
        "message": "Listen Alice, I will be honest, I want to try to have more meaningful conversations, I want to ask you a deep question so we skip the small talk, 'What TV show is on your bucket list?'"
    },
    {
        "time": "2021-11-18 22:36:30.254556",
        "user": "Alice",
        "message": "that ' s a good idea . i ' m a big fan of the walking dead . what about you ?"
    }
]

~~~~HISTORY~~~~


~~~~PROMPT~~~~

# This is a list of conversation starters with format (conversation starter, [list of topics])
conversation_starters = [("What is the meaning of 'The Good', the ideal state for humans?", ['philosophy']), ('What would the earth be like without any humans', ['ecology', 'philosophy']), ('What is your ide

KeyboardInterrupt: 

In [67]:
ws.close()